<a href="https://colab.research.google.com/github/alandgmendes/pdot_estudos/blob/main/estudos_pdot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pymupdf

In [2]:
import fitz  # PyMuPDF
import requests
from urllib.parse import quote
import re
from shapely.geometry import Point, Polygon
import geopandas as gpd
import pandas as pd

In [3]:
pdf_url = "https://presidenteolegario.mg.gov.br/uploads/files/files/2636/original/LEI COMPLEMENTAR Nº 094 de 27 de julho 2020- Aprova o Plano Diretor do Município de Presidente Olegário_Anexos.pdf"

# Adicionar cabeçalhos para simular uma solicitação de navegador
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8'
}

# Faça o pedido para obter o conteúdo do PDF
response = requests.get(pdf_url, headers=headers)

file_name = 'ANEXOS_DO_PLANO_DIRETOR.pdf'
if response.status_code == 200:
    with open(file_name, "wb") as pdf_file:
        pdf_file.write(response.content)
    print("PDF baixado com sucesso!")
else:
    print(f"Falha ao baixar o PDF. Código de status: {response.status_code}")
    print(f"Mensagem: {response.text}")


PDF baixado com sucesso!


In [4]:
def extrair_texto_pdf(url):
    texto_total = ""

    # Abre o arquivo PDF
    pdf_document = fitz.open(url)

    # Itera sobre as páginas e extrai o texto
    for pagina_num in range(pdf_document.page_count):
        pagina = pdf_document[pagina_num]
        texto_total += pagina.get_text()

    # Fecha o arquivo PDF
    pdf_document.close()

    return texto_total

In [5]:
result = extrair_texto_pdf(file_name)
print(result)

 
MUNICÍPIO DE PRESIDENTE OLEGÁRIO 
GABINETE DO PREFEITO 
 
 
Página 54 
 
 
 
 
 
 
 
 
 
 
 
 
MAPAS 
 
MUNICÍPIO DE PRESIDENTE OLEGÁRIO 
GABINETE DO PREFEITO 
 
 
Página 55 
Mapa 1- Macrozonas Municipais; 
 
 
MUNICÍPIO DE PRESIDENTE OLEGÁRIO 
GABINETE DO PREFEITO 
 
 
Página 56 
Mapa 2- Zonas Rurais; 
 
 
MUNICÍPIO DE PRESIDENTE OLEGÁRIO 
GABINETE DO PREFEITO 
 
 
Página 57 
Mapa 3- Perímetro urbano e zonas da Sede Municipal; 
 
 
MUNICÍPIO DE PRESIDENTE OLEGÁRIO 
GABINETE DO PREFEITO 
 
 
Página 58 
Mapa 4- Perímetro urbano e zonas do Núcleo de Ocupação Urbana de Santiago de Minas; 
 
 
MUNICÍPIO DE PRESIDENTE OLEGÁRIO 
GABINETE DO PREFEITO 
 
 
Página 59 
Mapa 5- Perímetro urbano e zonas do Núcleo de Ocupação Urbana de Cruzeiro da Prata; 
 
 
MUNICÍPIO DE PRESIDENTE OLEGÁRIO 
GABINETE DO PREFEITO 
 
 
Página 60 
Mapa 6- Perímetro urbano e zonas do Núcleo de Ocupação Urbana de Ponte Firme; 
 
 
MUNICÍPIO DE PRESIDENTE OLEGÁRIO 
GABINETE DO PREFEITO 
 
 
Página 61 
Mapa 7- Perímetr

In [32]:
import re
import pandas as pd
from shapely.geometry import Point

def extrair_coordenadas(descricao_nucleo):
    # Expressão regular para encontrar os pontos de coordenadas
    padrao_coordenadas = re.compile(r'Pt(\d+), de coordenadas N ([-+]?\d*\.\d+) m e E ([-+]?\d*\.\d+) m')

    # Encontrar todas as correspondências de coordenadas na descrição
    correspondencias_coordenadas = padrao_coordenadas.finditer(descricao_nucleo)

    # Lista para armazenar as coordenadas
    coordenadas = []

    # Iterar sobre as correspondências
    for correspondencia_coordenada in correspondencias_coordenadas:
        pt_numero = correspondencia_coordenada.group(1)
        coordenada_norte = float(correspondencia_coordenada.group(2))
        coordenada_leste = float(correspondencia_coordenada.group(3))

        # Adicionar à lista de coordenadas
        coordenadas.append((pt_numero, coordenada_norte, coordenada_leste))

    return coordenadas

def extrair_nucleos(texto):
    # Expressão regular para encontrar conjunto de coordenadas
    padrao_nucleo = re.compile(r'Pt(\d+), de coordenadas N ([-+]?\d*\.\d+) m e E ([-+]?\d*\.\d+) m')

    # Encontrar todas as correspondências de conjunto de coordenadas no texto
    correspondencias_nucleo = padrao_nucleo.finditer(texto)

    # Lista para armazenar os pontos de cada núcleo
    pontos_por_nucleo = []

    # Iterar sobre as correspondências de conjunto de coordenadas
    for correspondencia_nucleo in correspondencias_nucleo:
        pt_numero = correspondencia_nucleo.group(1)
        coordenada_norte = float(correspondencia_nucleo.group(2))
        coordenada_leste = float(correspondencia_nucleo.group(3))

        # Adicionar à lista de coordenadas
        pontos_por_nucleo.append((pt_numero, coordenada_norte, coordenada_leste))

    # Criar DataFrame com os pontos
    df_pontos = pd.DataFrame(pontos_por_nucleo, columns=['Pt', 'Norte', 'Leste'])

    # Criar coluna de geometry com objetos Point
    df_pontos['geometry'] = df_pontos.apply(lambda row: Point(row['Leste'], row['Norte']), axis=1)

    return df_pontos

# Testar a função com o novo trecho de texto
df_pontos = extrair_nucleos(result)

# Visualizar o DataFrame
print(df_pontos)


   Pt       Norte      Leste                      geometry
0  25  7995100.92  360849.90   POINT (360849.9 7995100.92)
1  26  7995213.66  360733.93  POINT (360733.93 7995213.66)
2  27  7995125.60  360649.72   POINT (360649.72 7995125.6)
3  33  7995240.18  360627.58  POINT (360627.58 7995240.18)
4  34  7995248.65  360649.63  POINT (360649.63 7995248.65)
5   0  7995393.17  360798.27  POINT (360798.27 7995393.17)
6  47  7990330.57  379170.32  POINT (379170.32 7990330.57)
7  48  7990425.02  379104.72  POINT (379104.72 7990425.02)
8  54  7990677.49  379389.25  POINT (379389.25 7990677.49)
9  55  7990701.62  379393.35  POINT (379393.35 7990701.62)
